## Content

- **Hyperparam**
    - ccp_alpha

- After Ensemble, can we make further improvements? - **Hyper-Parameter tuning**

- Can we add more randomness? - **Extra Trees or Extremely Randomised Trees**

- Let's test our understanding - Questions

- **Summary**

## Hyperparam

### **ccp_alpha- cost complexity pruning**



* **PRUNING: Sometimes after we make a tree using the greedy approach and maximising information gain at each step, we eventually end up with some redundant or very less usefull branches. Hence after the tree is completed, we can now go back and merge / remove some paths / subtress inside the tree making it simpler and effecient. This is called Pruning**
* This is basicaly used for pruning the base learners 
* We can control the overfitting and undefitting of the base learners using the value $α$, this is almost similar tp $λ$ whixh we used in linear and logistic regression
* so the idea here is to minimise the loss associated with the decision tree and $α$ times the number of terminal nodes (leaves) which controls overfitting 
 * min(loss + $α$ * number of leaves in the tree)
* As the depth of tree increases we know the loss decreases, where the number of leaf nodes in increases, this trade-off between the loss and number of leaves can be controlled using $α$ like regularisation.

<img src='https://drive.google.com/uc?id=1OS3oMsWLqUUzuKEO--J_5dlyDNknjMRl'>



<img src='https://drive.google.com/uc?id=1jMy3PfJkOZWRDvLHd2D6fh8FQEcgSpjy'>



<img src='https://drive.google.com/uc?id=1JsoH0fNs82BYPKOStZDT4uN2bygFaQbE'>


# **Hyper Parameter Tuning**
# **Can we make further improvements?**


### Grid Search

**What is GridSearch**
- "In grid search the set of trials is formed by assembling every possible combination of values"
- Basically, trying out every possible combination of hyperparameters in the search space.

So we define a parameter space, i.e the range of values for each parameter that we want to try, and then we write a computer program to try out each combination of hyperparamers and we pick the best one.

Note that this is a great example where we utilise parallel and even distributed computing at scale

<img src='https://drive.google.com/uc?id=1d_q60Sui-6Txb6LC-XQ2rzbIMpFzHJT8'>

In [ ]:
# Defining Parametes
params = {
          'n_estimators' : [100,200,300,400],
          'max_depth' : [3,5,10],
          'criterion' : ['gini', 'entropy'],
          'bootstrap' : [True, False],
          'max_features' : [8,9,10]
         }

Note that although we are trying out just a few possibilities for each parameter, in total it becomes many combinations:

> **4 * 3 * 2 * 2* 3 = 144**

Now we can use SK-Learn to try all these combinations


In [ ]:
from sklearn.model_selection import GridSearchCV

tuning_function = GridSearchCV(estimator = RandomForestClassifier(), 
                               param_grid = params,
                               scoring = 'accuracy',
                               cv = 3,
                               n_jobs=-1
                               )

In [ ]:
# Now we will fit all combinations, this will take some time to run. (5-6 mins)
tuning_function.fit(X_sm, y_sm)

parameters = tuning_function.best_params_
score = tuning_function.best_score_
print(parameters)
print(score)  

{'bootstrap': False, 'criterion': 'gini', 'max_depth': 10, 'max_features': 8, 'n_estimators': 200}
0.9063852813852814


- Remeber that Grid Search is a **brute force search**


**Why Grid Search?**
- Grid Search performs really well for small dimensions of dataset
- It's appropriate for small and quick searches of hyperparameter values that are known to perform well generally.
- It’s exhaustive and leaves no stone unturned

**Cons of Grid Search**
- *Curse of Dimensionality*: 
  - Let's say there are 7 hyperparameters and each hyperparameter has 5 possible values.
  - Using Grid Search, no. of combinations tried will be 5 raised to 7, which is 78125 
- This exhaustive approach of "Just try everything" becomes compute intesive very quickly. 
- It doesn't work well when the number of hyperparameters is relatively large.

In [ ]:
from sklearn.model_selection import KFold, cross_validate

tree_clf = RandomForestClassifier(random_state=7, bootstrap=False, criterion='gini', max_depth=10, max_features=8, n_estimators=200)
kfold = KFold(n_splits=10)
cv_acc_results = cross_validate(tree_clf, X_sm, y_sm, cv = kfold, scoring = 'accuracy', return_train_score = True)

print(f"K-Fold Accuracy Mean: Train: {cv_acc_results['train_score'].mean()*100} Validation: {cv_acc_results['test_score'].mean()*100}")
print(f"K-Fold Accuracy Std: Train: {cv_acc_results['train_score'].std()*100} Validation: {cv_acc_results['test_score'].std()*100}")

K-Fold Accuracy Mean: Train: 99.86771618715021 Validation: 91.72767332549941
K-Fold Accuracy Std: Train: 0.09621708717668379 Validation: 5.510701433405571


**Great, wo do see a significant improvement in performance.**
- Train Set: **99.9 %**
- Test Set: **92 %**

Further notes
- Again, this does seem an overfit case, maybe we can reduce max depth a bit. 
- Grid search can be a great starting point since it eleminates many combinations, we can still do some manual hand-tuning based on domain knowledge of trade-offs, business priorities and required precission/recall levels.

## **Randomized Search**

Just like Grid Search which is an exhaustive brute for search, we can use a Random Search as well, which will try hyper-parameters randomly from a finite list of options, or from a distribution.

It can be used when you want to try a hyper-parameter within a certain range with some asssociated probability

<img src='https://drive.google.com/uc?id=18ZDbe9f-6equ7aOdXSVB046Zcvj1UoUa'>


<img src='https://drive.google.com/uc?id=1MFKfYuLLYD42_IHJd7AtSrKyrCle8Ouf'>

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform


params = {'ccp_alpha': uniform(loc=0, scale=0.4)}  # sample from uniform dist between 0 to (0+0.4)=0.4

tuning_function = RandomizedSearchCV(estimator = RandomForestClassifier(random_state=7, bootstrap=False, criterion='gini', 
                                                                  max_depth=10, max_features=8, n_estimators=200), 
                               param_distributions = params,  # notice arg changeed from param_grid to param_distributions
                               scoring = 'accuracy',
                               cv = 3,
                               n_iter=15,  # Number of times to run random combination
                               n_jobs=-1
                               )

tuning_function.fit(X_sm, y_sm)

parameters = tuning_function.best_params_
score = tuning_function.best_score_
print(parameters)
print(score)  

{'ccp_alpha': 0.05604253214986166}
0.7159090909090908


Here as we can see, we ran a very small randomized seach for just one hyperparmeter, and the score is reduced. So we will move ahead with our original value for *ccp_alpha* which was defaulted to 0

**Benifits of Randomized Search**
- Hyperparameter space is explored more widely, especially for more important variables
- No need to provide an explicit set of possible values for every hyperparameter
  - If we provide a good prior (knowlegde of where there is higher chance of finding the best value)
- Best configuration can be found in fewer iterations

**Short Comings of Random Search**
- It does NOT use information from previous experiments to select the next set.
- Each next guess is independent of previous experiment

# **Coarse to Fine Tuning**

As we can see searching for the right hyperparameter is expensive, and it is not possible (or too expensive) in most cases to find the right parameter by using an optimisation framework.

So here we can use clever systematic approaches to make our seach slighly more "guided".

- In coarse to fine approach we first try a wider range (coarse) of a give hyperparameter.
- Then, once we have the best one, we try values near to that best value in smaller intervals (Fine).
- We can repeat this, until we keep getting reasonable improvements.


For example, lets try to tune the n_estimators parameter again

<img src='https://drive.google.com/uc?id=1JvyopaI7uCCHegzdTd_ST3LvhKKm3AdU'>

<img src='https://drive.google.com/uc?id=1i9P88jgM3ivBsG01nreWSxzRewMbx36T'>

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'n_estimators': [100,  200, 300, 400, 500]}  # sample from uniform dist between 0 to (0+0.4)=0.4

tuning_function = GridSearchCV(estimator = RandomForestClassifier(random_state=7, bootstrap=False, criterion='gini', 
                                                                  max_depth=10, max_features=8, n_estimators=200), 
                               param_grid = params,  # notice arg changeed from param_grid to param_distributions
                               scoring = 'accuracy',
                               cv = 3,
                               n_jobs=-1
                               )

tuning_function.fit(X_sm, y_sm)

parameters = tuning_function.best_params_
score = tuning_function.best_score_
print(parameters)
print(score)  

{'n_estimators': 400}
0.9042207792207794


We see 2 things:
- Best value for n_estimators is 400
- When we used a full grid seach it was 200. This means that hyperparamets cannot be (ideally) tuned independently


Now we can look in the vicinity of "400" with finer pertebations.

In [ ]:
params = {'n_estimators': [350, 370, 400, 420, 450]}  # sample from uniform dist between 0 to (0+0.4)=0.4

tuning_function = GridSearchCV(estimator = RandomForestClassifier(random_state=7, bootstrap=False, criterion='gini', 
                                                                  max_depth=10, max_features=8, n_estimators=200), 
                               param_grid = params,  # notice arg changeed from param_grid to param_distributions
                               scoring = 'accuracy',
                               cv = 3,
                               n_jobs=-1
                               )

tuning_function.fit(X_sm, y_sm)

parameters = tuning_function.best_params_
score = tuning_function.best_score_
print(parameters)
print(score)  

{'n_estimators': 400}
0.9042207792207794


We can see that, 400 is the best.
- We can try more from 370 to 420 with intervals of say 10.
- We can stop here if we are satisfied. (For this lecture we will stop here)

<hr>
<hr>
<hr>
<hr>
<hr>
<hr>

## Extra Trees or Extremely Randomised Trees 


let's compare these with Random Forest

**What do we do in random forest?**
* We do random row sampling and column sampling and then we do aggregation, in which this randomisation and aggregation play a key role in reducing the variace keeping the bias similar, further avoiding overfitting 

**Randomisation is a great, useful and very powerful strategy for Regularization**
 



<img src='https://drive.google.com/uc?id=1DSnJpwOzFCuyQdF5KsUDktJPYZ_szZdv'>


### **What do extra trees do?**

In extra tree does random row and column sampling and aggregation just like decision trees but it also randomly picks the threshold (τ) to split for numerical features.

* In decsision trees we saw using the features values as threshold we calculate information gain and then choose the thresold of the split , basing on the values of information gain

* In an extra tree we choose this threshold (τ) also randomly ,adding one more randomisation on top of random forest,
 * say there are $m$ rows in a feature we randomly select few rows and set the threshold basing on their IG values 
*This is useful when the $m$ is large, but these require more base laerners as the trees are not perfect 
* Hence these are not widely used



<img src='https://drive.google.com/uc?id=1hLXJNCgbUqp3emjQineZvLf8W6dKfwnV'>



<img src='https://drive.google.com/uc?id=1GncvBjGn0p0ZdEYl49SaZxNSgZMIgY1Y'>



## Questions


**What if the models have high bias in an ensemble?**

* We know Random forest is an ensemble of high varaince models, that is slightly overgfitting models
* We see the Gradient Boosted Decision Tree which applies boosting, which deals with high bias or underfitting models


<img src='https://drive.google.com/uc?id=1nAOfG-cO5tV71OaEZWI4t9CgUBTLPA71'>


**What if ensemble a deep decision tree and a simple linear classifier?**

* This doesn't work as expected beacuse the random forest expects highy variance models, as it does aggregation at the end which reduces the variancce, but do not effect the bias.Due to which if a model of high bias is there, the high bias remains the same which we should avoid.


<img src='https://drive.google.com/uc?id=1lAQIpDbsaoAvUrcVlRduRdq8xpE4GfCb'>




## **What happens when $m$ is very close to $n$ ?**
* When $m$ is close to $n$ the base learners overfit on the whole train data set. Due to which Aggregation doesn't work in reducing the variance 
* So, when $\frac{m}{n}$ increases, Random forest overfits.
* But whwn $\frac{m}{n}$ decreases the base leraners overfit, due to which we require more number of trees for to reduce the variance.


<img src='https://drive.google.com/uc?id=1XABn5maJst8ugowX4fX4FJwYUbw1jYnV'>


**So here, Should we take weighted average of the base models in aggregation ?**

No, this is not required beacuse we are taking Base learners with equal weight 



<img src='https://drive.google.com/uc?id=1Q5mVqPrdxHw6i88VFZrRAop9pvPz1o86'>



* We saw that aggregation is the key in Random forest ,

* But we might have two different types of Base learners in the random forest 
 * With high variance and low bias or
 * With medium varaince and medium bias 
* Here as we do aggregation both the models become
 * with low variance and low bias and 
 * with low variance and medium bias respectively

So it's better to have high variance and low bias base learners.




<img src='https://drive.google.com/uc?id=10eSeKSrmQsNc3jLmqhxLDFxJ7bsETR6X'>

## Summary


* Random forest works well when  Dimensionality is not too large
* As it is trivially parallelizable, it works very well even when the $n$ is large
* The hyper parameters in Random forest are 
 * $k$ : the number of trees
 * $\frac{m}{n}$ ratio and
 
 * $\frac{d'}{d}$ ratio


<img src='https://drive.google.com/uc?id=1Tm-8-_Fnp_V0fpCaqe7CzXjMFRfJ7QY5'>
